# CS178 Project

## Import part

In [1]:
import os
import numpy as np
import pandas as pd
import cv2

from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

If using Colab, uncomment and run the following session:

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# print(os.getcwd())
# os.chdir('/content/drive/MyDrive/CS178')
# print(os.getcwd())
# print(os.listdir())

## Function Define

In [3]:
class ConvNet(nn.Module):
    def __init__(self, label_size):
        super().__init__()
        # Pooling Layer
        self.pool = nn.MaxPool2d(2, 2)

        # Convolutional Layer
        # (in_channels, out_channels, kernel_size)
        # first in channel is 1 because its greyscale
        self.conv1 = nn.Conv2d(1, 6, 5) # (6, 350, 350) -> (6, 173, 173)
        self.conv2 = nn.Conv2d(6, 16, 5) # (6, 173, 173) -> (16, 84, 84)

        # Linear Layer
        self.fc1 = nn.Linear(16 * 84 * 84, 120) # shape of image after pooling twice (16, 84, 84)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, label_size)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
def read_pic(data, categories):
    folder_path = 'images/'
    image_paths = data.iloc[:, 1]
    labels = data.iloc[:, 2].str.lower()
    X = []
    label_to_int = {category: i for i, category in enumerate(categories)}
    integer_labels = labels.map(label_to_int)
    y = torch.LongTensor(integer_labels.to_numpy())
    for filename in image_paths:
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Check for common image file extensions
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (350, 350))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                X.append(torch.Tensor(img))
    X = torch.stack(X).unsqueeze(1) # convert to pytorch tensor (batch, channels, w, h)
    return X, y

In [5]:
def train_process(myDevice, myCNN, loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(myCNN.parameters(), lr=0.001, momentum=0.9)
    for epoch in range(2):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(loader, 0):
            inputs, labels = data[0].to(myDevice), data[1].to(myDevice)

            optimizer.zero_grad()
            outputs = myCNN(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 100 == 99:    # print every 100 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
    print('Finished Training')
    return myCNN

In [6]:
def accuracy(myDevice, myCNN, loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data[0].to(myDevice), data[1].to(myDevice)
            # calculate outputs by running images through the network
            outputs = myCNN(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            return (correct / total)

## Test Part

### Check if GPU is available

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cpu


### Import images

In [8]:
categories = [
    "anger",
    "surprise",
    "disgust",
    "fear",
    "neutral",
    "happiness",
    "sadness",
    "contempt"
]

X, y = read_pic(pd.read_csv('data/legend.csv'), categories)
# X = X[:1000]
# y = y[:1000]
seed = 1234
Xtr, Xva, ytr, yva = train_test_split(X, y, test_size=0.2, random_state=seed)

torch.Size([800, 1, 350, 350])
torch.Size([200, 1, 350, 350])
tensor([5, 4, 4, 1, 5, 5, 6, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 5, 4, 4, 5, 4,
        4, 5, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 5, 4, 5, 4, 5, 4, 4, 4, 5, 5, 5, 5,
        4, 4, 5, 4, 5, 5, 4, 4, 4, 6, 4, 5, 5, 4, 5, 5, 4, 4, 5, 5, 5, 4, 5, 4,
        4, 5, 4, 5, 5, 5, 4, 4, 4, 5, 4, 4, 4, 5, 4, 4, 5, 4, 5, 4, 4, 4, 7, 4,
        4, 5, 5, 4, 4, 5, 4, 4, 5, 5, 5, 4, 5, 5, 4, 5, 4, 6, 4, 4, 5, 4, 5, 5,
        4, 4, 4, 4, 4, 5, 4, 4, 4, 5, 1, 4, 6, 5, 5, 5, 4, 4, 4, 1, 4, 6, 5, 5,
        4, 5, 4, 5, 4, 4, 4, 4, 5, 5, 4, 5, 4, 4, 4, 5, 4, 4, 4, 5, 5, 4, 5, 4,
        5, 5, 5, 4, 4, 5, 5, 4, 5, 5, 4, 4, 4, 5, 5, 4, 5, 4, 5, 4, 5, 4, 4, 4,
        4, 4, 5, 4, 4, 0, 5, 5, 4, 4, 5, 4, 5, 5, 5, 5, 4, 4, 4, 5, 4, 5, 4, 5,
        4, 4, 4, 4, 5, 4, 5, 4, 4, 5, 4, 4, 4, 4, 4, 4, 5, 5, 4, 4, 4, 4, 5, 5,
        5, 4, 4, 4, 4, 5, 5, 1, 6, 5, 4, 5, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 5, 5, 5, 4, 4, 5, 4, 4, 5, 4, 5, 4, 5, 4, 4,

In [13]:
print(Xtr.shape)
print(Xva.shape)
print(ytr)
print(yva)

batch_size = 4
train_dataset = torch.utils.data.TensorDataset(Xtr, ytr)
test_dataset = torch.utils.data.TensorDataset(Xva, yva)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

torch.Size([800, 1, 350, 350])
torch.Size([200, 1, 350, 350])
tensor([5, 4, 4, 1, 5, 5, 6, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 5, 4, 4, 5, 4,
        4, 5, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 5, 4, 5, 4, 5, 4, 4, 4, 5, 5, 5, 5,
        4, 4, 5, 4, 5, 5, 4, 4, 4, 6, 4, 5, 5, 4, 5, 5, 4, 4, 5, 5, 5, 4, 5, 4,
        4, 5, 4, 5, 5, 5, 4, 4, 4, 5, 4, 4, 4, 5, 4, 4, 5, 4, 5, 4, 4, 4, 7, 4,
        4, 5, 5, 4, 4, 5, 4, 4, 5, 5, 5, 4, 5, 5, 4, 5, 4, 6, 4, 4, 5, 4, 5, 5,
        4, 4, 4, 4, 4, 5, 4, 4, 4, 5, 1, 4, 6, 5, 5, 5, 4, 4, 4, 1, 4, 6, 5, 5,
        4, 5, 4, 5, 4, 4, 4, 4, 5, 5, 4, 5, 4, 4, 4, 5, 4, 4, 4, 5, 5, 4, 5, 4,
        5, 5, 5, 4, 4, 5, 5, 4, 5, 5, 4, 4, 4, 5, 5, 4, 5, 4, 5, 4, 5, 4, 4, 4,
        4, 4, 5, 4, 4, 0, 5, 5, 4, 4, 5, 4, 5, 5, 5, 5, 4, 4, 4, 5, 4, 5, 4, 5,
        4, 4, 4, 4, 5, 4, 5, 4, 4, 5, 4, 4, 4, 4, 4, 4, 5, 5, 4, 4, 4, 4, 5, 5,
        5, 4, 4, 4, 4, 5, 5, 1, 6, 5, 4, 5, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 5, 5, 5, 4, 4, 5, 4, 4, 5, 4, 5, 4, 5, 4, 4,

### Train a CNN

In [14]:
net = ConvNet(8).to(device)
print(net)
# To see that our CNN really improved after training, we will give training error and testing error first
print(f'Training Accuracy: {accuracy(device, net, trainloader) * 100} %')
print(f'Testing Accuracy: {accuracy(device, net, testloader) * 100} %')
net = train_process(device, net, trainloader)

ConvNet(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=112896, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=8, bias=True)
)
Training Accuracy: 25.0 %
Testing Accuracy: 25.0 %
[1,   100] loss: 11.186
[1,   200] loss: 0.946
[2,   100] loss: 0.952
[2,   200] loss: 1.022
Finished Training


In [15]:
PATH = './test.pth'
torch.save(net.state_dict(), PATH)

In [16]:
net = ConvNet(8).to(device)
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

### Check Error rate

In [17]:
print(f'Training Accuracy: {accuracy(device, net, trainloader) * 100} %')
print(f'Testing Accuracy: {accuracy(device, net, testloader) * 100} %')

Training Accuracy: 50.0 %
Testing Accuracy: 75.0 %
